In [1]:
import numpy as np

### Простой метод Гаусса

In [2]:
import numpy as np

def gauss_solve(matrix, rhs):
    """
    Решение системы линейных уравнений методом Гаусса (без выбора главного элемента).
    
    Аргументы:
    matrix : numpy.ndarray
        Матрица коэффициентов (должна быть квадратной).
    rhs : numpy.ndarray
        Вектор свободных членов.
    
    Возвращает:
    numpy.ndarray
        Решение системы уравнений.
    """
    matrix = matrix.astype(float)
    rhs = rhs.astype(float)
    size = len(rhs)
    
    # Прямой ход
    for pivot in range(size):
        if abs(matrix[pivot, pivot]) < 1e-10:
            raise ValueError("Опорный элемент слишком мал или равен нулю.")
        for row in range(pivot + 1, size):
            multiplier = matrix[row, pivot] / matrix[pivot, pivot]
            matrix[row, pivot:] -= multiplier * matrix[pivot, pivot:]
            rhs[row] -= multiplier * rhs[pivot]
    
    # Обратная подстановка
    solution = np.zeros(size)
    for idx in range(size - 1, -1, -1):
        temp_sum = np.dot(matrix[idx, idx + 1:], solution[idx + 1:])
        solution[idx] = (rhs[idx] - temp_sum) / matrix[idx, idx]
    
    return solution

In [3]:
A = np.array([[2, 1, -1],
                [-3, -1, 2],
                [-2, 1, 2]])
b = np.array([8, -11, -3])
try:
    solution = gauss_solve(A, b)
    print("Решение системы (простой метод Гаусса):", solution)
except ValueError as e:
    print("Ошибка:", e)

Решение системы (простой метод Гаусса): [ 2.  3. -1.]


### Метод Жордана-Гаусса

In [4]:
import numpy as np

def solve_by_gauss_jordan(coeffs, constants):
    """
    Метод Жордана-Гаусса для решения системы линейных уравнений.
    
    Аргументы:
    coeffs : numpy.ndarray
        Квадратная матрица коэффициентов.
    constants : numpy.ndarray
        Вектор свободных членов.
    
    Возвращает:
    numpy.ndarray
        Вектор решений системы.
    
    Исключения:
    ValueError: При вырожденности матрицы или почти нулевом ведущем элементе.
    """
    coeffs = coeffs.astype(float)
    constants = constants.astype(float)
    dim = len(constants)

    # Объединяем коэффициенты и свободные члены в расширенную матрицу
    augmented = np.concatenate((coeffs, constants.reshape(-1, 1)), axis=1)

    for col in range(dim):
        # Поиск строки с максимальным значением по модулю в текущем столбце
        pivot_row = col + np.argmax(np.abs(augmented[col:, col]))
        augmented[[col, pivot_row]] = augmented[[pivot_row, col]]

        # Проверка на вырождение
        if abs(augmented[col, col]) < 1e-10:
            raise ValueError("Система не имеет единственного решения или матрица вырождена.")
        
        # Нормализация текущей строки
        augmented[col] /= augmented[col, col]

        # Обнуление всех остальных значений в текущем столбце
        for row in range(dim):
            if row != col:
                coeff = augmented[row, col]
                augmented[row] -= coeff * augmented[col]

    # Последний столбец содержит решение
    return augmented[:, -1]

In [5]:

# Задаем систему уравнений:
A = np.array([[2, 1, -1],
                [-3, -1, 2],
                [-2, 1, 2]])
b = np.array([8, -11, -3])

try:
    solution = solve_by_gauss_jordan(A, b)
    print("Решение системы (метод Жордана-Гаусса):", solution)
except ValueError as e:
    print("Ошибка:", e)

Решение системы (метод Жордана-Гаусса): [ 2.  3. -1.]


### Метод Гаусса с методом исключения

In [6]:
import numpy as np

def pivot_gauss_solver(mat, vec):
    """
    Решение системы линейных уравнений методом Гаусса с выбором главного элемента.
    
    Аргументы:
    mat : numpy.ndarray
        Квадратная матрица коэффициентов.
    vec : numpy.ndarray
        Вектор свободных членов.
    
    Возвращает:
    numpy.ndarray
        Решение системы в виде одномерного массива.
    """
    mat = mat.astype(float)
    vec = vec.astype(float)
    size = len(vec)

    # Прямой ход с частичным выбором ведущего элемента
    for step in range(size):
        # Нахождение строки с наибольшим по модулю элементом в текущем столбце
        lead = step + np.argmax(np.abs(mat[step:, step]))
        # Перестановка строк для повышения устойчивости
        mat[[step, lead]] = mat[[lead, step]]
        vec[step], vec[lead] = vec[lead], vec[step]

        if abs(mat[step, step]) < 1e-10:
            raise ValueError("Невозможно решить систему: матрица почти вырождена.")

        # Преобразование строк ниже
        for row in range(step + 1, size):
            coeff = mat[row, step] / mat[step, step]
            mat[row, step:] -= coeff * mat[step, step:]
            vec[row] -= coeff * vec[step]

    # Обратная подстановка
    result = np.zeros(size)
    for idx in range(size - 1, -1, -1):
        subtotal = np.dot(mat[idx, idx + 1:], result[idx + 1:])
        result[idx] = (vec[idx] - subtotal) / mat[idx, idx]

    return result

In [7]:
A = np.array([[2, 1, -1],
                [-3, -1, 2],
                [-2, 1, 2]])
b = np.array([8, -11, -3])
try:
    solution = pivot_gauss_solver(A, b)
    print("Решение системы (с выбором опорного элемента):", solution)
except ValueError as e:
    print("Ошибка:", e)

Решение системы (с выбором опорного элемента): [ 2.  3. -1.]


### Метод LU-Разложения

In [12]:
def lu_decomposition(A):
    """
    Выполняет LU-разложение матрицы A методом Дулитла.
    Предполагается, что A - квадратная матрица и ее LU-разложение существует.
    
    Возвращает:
        L : нижнетреугольная матрица с единичными элементами на диагонали.
        U : верхнетреугольная матрица.
    
    Исключение:
        ValueError, если на диагонали U оказывается элемент, близкий к нулю.
    """
    A = A.astype(float)
    n = A.shape[0]
    L = np.eye(n, dtype=float)  # Инициализируем L как единичную матрицу
    U = np.zeros((n, n), dtype=float)

    for i in range(n):
        # Вычисляем i-ю строку U
        for j in range(i, n):
            sum_u = sum(L[i, k] * U[k, j] for k in range(i))
            U[i, j] = A[i, j] - sum_u
        
        # Проверка на нулевой опорный элемент в U[i, i]
        if abs(U[i, i]) < 1e-12:
            raise ValueError("Нулевой опорный элемент обнаружен. LU-разложение невозможно.")
        
        # Вычисляем i-й столбец L
        for j in range(i+1, n):
            sum_l = sum(L[j, k] * U[k, i] for k in range(i))
            L[j, i] = (A[j, i] - sum_l) / U[i, i]
    
    return L, U

def forward_substitution(L, b):
    """
    Решает систему Ly = b методом прямой подстановки.
    
    Возвращает:
        y : вектор-решение.
    """
    n = L.shape[0]
    y = np.zeros(n, dtype=float)
    for i in range(n):
        y[i] = b[i] - sum(L[i, j] * y[j] for j in range(i))
    return y

def backward_substitution(U, y):
    """
    Решает систему Ux = y методом обратной подстановки.
    
    Возвращает:
        x : вектор-решение.
    """
    n = U.shape[0]
    x = np.zeros(n, dtype=float)
    for i in reversed(range(n)):
        x[i] = (y[i] - sum(U[i, j] * x[j] for j in range(i+1, n))) / U[i, i]
    return x

def lu_solve(A, b):
    """
    Решает систему линейных уравнений Ax = b с помощью LU-разложения.
    
    Параметры:
        A : numpy.ndarray
            Квадратная матрица коэффициентов.
        b : numpy.ndarray
            Вектор правых частей.
    
    Возвращает:
        x : numpy.ndarray
            Вектор решения системы.
    """
    L, U = lu_decomposition(A)
    y = forward_substitution(L, b)
    x = backward_substitution(U, y)
    return x

In [13]:
# Определяем матрицу коэффициентов A и вектор правых частей b
A = np.array([[2, 1, -1],
                [-3, -1, 2],
                [-2, 1, 2]])
b = np.array([8, -11, -3])

try:
    solution = lu_solve(A, b)
    print("Решение системы (LU-разложение):", solution)
except ValueError as e:
    print("Ошибка:", e)

Решение системы (LU-разложение): [ 2.  3. -1.]
